<a href="https://colab.research.google.com/github/SimonGeb/Deep-Learning/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/TU Delft/Master/Deep Learning/Reproducability Project

/content/drive/MyDrive/TU Delft/Master/Deep Learning/Reproducability Project


In [ ]:
pip install --upgrade tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install rawpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# uniform content loss + adaptive threshold + per_class_input + recursive G
# improvement upon cqf37
from __future__ import division
import os, time, scipy.io
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tf_slim as slim
import numpy as np
import rawpy
import glob
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# set directories here
input_dir = './dataset/Sony/short/'
gt_dir = './dataset/Sony/long/'
checkpoint_dir = './checkpoint/Own_RMSprop/'
result_dir = './result_Own_RMSprop/'

# get train IDs
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]
print(train_ids)
print(len(train_ids))
ps = 512  # patch size for training
save_freq = 500
num_epochs = 4000

[200, 224, 14, 124, 121, 41, 225, 123, 112, 144, 151, 58, 174, 104, 113, 206, 95, 23, 29, 154, 4, 230, 110, 26, 114, 138, 143, 65, 62, 100, 223, 31, 66, 119, 161, 134, 43, 186, 204, 70, 51, 85, 159, 189, 175, 90, 194, 231, 137, 132, 53, 135, 214, 165, 219, 155, 75, 212, 78, 182, 180, 205, 128, 102, 190, 202, 118, 133, 131, 10, 166, 216, 88, 136, 108, 99, 57, 48, 146, 148, 197, 184, 84, 209, 117, 96, 13, 164, 141, 56, 92, 109, 160, 19, 73, 12, 218, 207, 15, 27, 37, 94, 59, 71, 142, 60, 2, 9, 122, 152, 97, 49, 127, 130, 18, 64, 46, 181, 52, 150, 47, 83, 50, 36, 158, 232, 171, 39, 222, 72, 156, 86, 44, 42, 173, 169, 91, 145, 215, 179, 63, 149, 67, 221, 21, 168, 81, 196, 220, 1, 33, 129, 38, 183, 195, 76, 98, 28, 157, 17, 24]
161


In [ ]:
#set different debug schemes
DEBUG = None 
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]
    num_epochs = 10
if DEBUG == 2:
    save_freq = 5
    train_ids = train_ids[0:40]
    num_epochs = 15

In [ ]:
def lrelu(x):
    return tf.maximum(x * 0.2, x)

def bytescale(data, cmin=None, cmax=None, high=255, low=0):
    """
    Byte scales an array (image).
    Byte scaling means converting the input image to uint8 dtype and scaling
    the range to ``(low, high)`` (default 0-255).
    If the input image already has dtype uint8, no scaling is done.
    This function is only available if Python Imaging Library (PIL) is installed.
    """
    if data.dtype == np.uint8:
        return data

    if high > 255:
        raise ValueError("`high` should be less than or equal to 255.")
    if low < 0:
        raise ValueError("`low` should be greater than or equal to 0.")
    if high < low:
        raise ValueError("`high` should be greater than or equal to `low`.")

    if cmin is None:
        cmin = data.min()
    if cmax is None:
        cmax = data.max()

    cscale = cmax - cmin
    if cscale < 0:
        raise ValueError("`cmax` should be larger than `cmin`.")
    elif cscale == 0:
        cscale = 1

    scale = float(high - low) / cscale
    bytedata = (data - cmin) * scale + low
    return (bytedata.clip(low, high) + 0.5).astype(np.uint8)


def toimage(arr, high=255, low=0, cmin=None, cmax=None, pal=None,
            mode=None, channel_axis=None):
    """Takes a numpy array and returns a PIL image.
    """
    data = np.asarray(arr)
    if np.iscomplexobj(data):
        raise ValueError("Cannot convert a complex-valued array.")
    shape = list(data.shape)
    valid = len(shape) == 2 or ((len(shape) == 3) and
                                ((3 in shape) or (4 in shape)))
    if not valid:
        raise ValueError("'arr' does not have a suitable array shape for "
                         "any mode.")
    if len(shape) == 2:
        shape = (shape[1], shape[0])  # columns show up first
        if mode == 'F':
            data32 = data.astype(np.float32)
            image = Image.frombytes(mode, shape, data32.tostring())
            return image
        if mode in [None, 'L', 'P']:
            bytedata = bytescale(data, high=high, low=low,
                                 cmin=cmin, cmax=cmax)
            image = Image.frombytes('L', shape, bytedata.tostring())
            if pal is not None:
                image.putpalette(np.asarray(pal, dtype=np.uint8).tostring())
                # Becomes a mode='P' automagically.
            elif mode == 'P':  # default gray-scale
                pal = (np.arange(0, 256, 1, dtype=np.uint8)[:, np.newaxis] *
                       np.ones((3,), dtype=np.uint8)[np.newaxis, :])
                image.putpalette(np.asarray(pal, dtype=np.uint8).tostring())
            return image
        if mode == '1':  # high input gives threshold for 1
            bytedata = (data > high)
            image = Image.frombytes('1', shape, bytedata.tostring())
            return image
        if cmin is None:
            cmin = np.amin(np.ravel(data))
        if cmax is None:
            cmax = np.amax(np.ravel(data))
        data = (data*1.0 - cmin)*(high - low)/(cmax - cmin) + low
        if mode == 'I':
            data32 = data.astype(np.uint32)
            image = Image.frombytes(mode, shape, data32.tostring())
        else:
            raise ValueError(_errstr)
        return image

    # if here then 3-d array with a 3 or a 4 in the shape length.
    # Check for 3 in datacube shape --- 'RGB' or 'YCbCr'
    if channel_axis is None:
        if (3 in shape):
            ca = np.flatnonzero(np.asarray(shape) == 3)[0]
        else:
            ca = np.flatnonzero(np.asarray(shape) == 4)
            if len(ca):
                ca = ca[0]
            else:
                raise ValueError("Could not find channel dimension.")
    else:
        ca = channel_axis

    numch = shape[ca]
    if numch not in [3, 4]:
        raise ValueError("Channel axis dimension is not valid.")

    bytedata = bytescale(data, high=high, low=low, cmin=cmin, cmax=cmax)
    if ca == 2:
        strdata = bytedata.tostring()
        shape = (shape[1], shape[0])
    elif ca == 1:
        strdata = np.transpose(bytedata, (0, 2, 1)).tostring()
        shape = (shape[2], shape[0])
    elif ca == 0:
        strdata = np.transpose(bytedata, (1, 2, 0)).tostring()
        shape = (shape[2], shape[1])
    if mode is None:
        if numch == 3:
            mode = 'RGB'
        else:
            mode = 'RGBA'

    if mode not in ['RGB', 'RGBA', 'YCbCr', 'CMYK']:
        raise ValueError(_errstr)

    if mode in ['RGB', 'YCbCr']:
        if numch != 3:
            raise ValueError("Invalid array shape for mode.")
    if mode in ['RGBA', 'CMYK']:
        if numch != 4:
            raise ValueError("Invalid array shape for mode.")

    # Here we know data and mode is correct
    image = Image.frombytes(mode, shape, strdata)
    return image

def upsample_and_concat(x1, x2, output_channels, in_channels):
    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal([pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2), strides=[1, pool_size, pool_size, 1])

    deconv_output = tf.concat([deconv, x2], 3)
    deconv_output.set_shape([None, None, None, output_channels * 2])

    return deconv_output


def network(input):
    conv1 = slim.conv2d(input, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_1')
    conv1 = slim.conv2d(conv1, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], padding='SAME')

    conv2 = slim.conv2d(pool1, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_1')
    conv2 = slim.conv2d(conv2, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv2_2')
    pool2 = slim.max_pool2d(conv2, [2, 2], padding='SAME')

    conv3 = slim.conv2d(pool2, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_1')
    conv3 = slim.conv2d(conv3, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv3_2')
    pool3 = slim.max_pool2d(conv3, [2, 2], padding='SAME')

    conv4 = slim.conv2d(pool3, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_1')
    conv4 = slim.conv2d(conv4, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv4_2')
    pool4 = slim.max_pool2d(conv4, [2, 2], padding='SAME')

    conv5 = slim.conv2d(pool4, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_1')
    conv5 = slim.conv2d(conv5, 512, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv5_2')

    up6 = upsample_and_concat(conv5, conv4, 256, 512)
    conv6 = slim.conv2d(up6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_1')
    conv6 = slim.conv2d(conv6, 256, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv6_2')

    up7 = upsample_and_concat(conv6, conv3, 128, 256)
    conv7 = slim.conv2d(up7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_1')
    conv7 = slim.conv2d(conv7, 128, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv7_2')

    up8 = upsample_and_concat(conv7, conv2, 64, 128)
    conv8 = slim.conv2d(up8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_1')
    conv8 = slim.conv2d(conv8, 64, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv8_2')

    up9 = upsample_and_concat(conv8, conv1, 32, 64)
    conv9 = slim.conv2d(up9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_1')
    conv9 = slim.conv2d(conv9, 32, [3, 3], rate=1, activation_fn=lrelu, scope='g_conv9_2')

    conv10 = slim.conv2d(conv9, 12, [1, 1], rate=1, activation_fn=None, scope='g_conv10')
    out = tf.depth_to_space(conv10, 2)
    return out


def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    black = raw.black_level_per_channel
    im = np.maximum((im - np.mean(black)), 0) / ((16383) - np.mean(black))  # subtract the black level
  
    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
  
    H = img_shape[0]
    W = img_shape[1]
    if (raw.raw_pattern == ([[1, 0], [2, 3]])).all():
      G = im[0:H:2, 0:W:2, :] # Every alternating value starting from position (0,0) 
      R = im[0:H:2, 1:W:2, :] # Every alternating value starting from position (0,1) 
      G_e = im[1:H:2, 1:W:2, :] # Every alternating value starting from position (1,1) 
      B = im[1:H:2, 0:W:2, :] # Every alternating value starting from position (1,0)
    if (raw.raw_pattern == ([[0, 1], [3, 2]])).all():
      R = im[0:H:2, 0:W:2, :] # Every alternating value starting from position (0,0) 
      G = im[0:H:2, 1:W:2, :] # Every alternating value starting from position (0,1) 
      B = im[1:H:2, 1:W:2, :] # Every alternating value starting from position (1,1) 
      G_e = im[1:H:2, 0:W:2, :] # Every alternating value starting from position (1,0)  
    if (raw.raw_pattern == ([[1, 2], [0, 3]])).all():
      G = im[0:H:2, 0:W:2, :] # Every alternating value starting from position (0,0) 
      B = im[0:H:2, 1:W:2, :] # Every alternating value starting from position (0,1) 
      G_e = im[1:H:2, 1:W:2, :] # Every alternating value starting from position (1,1) 
      R = im[1:H:2, 0:W:2, :] # Every alternating value starting from position (1,0)
    if (raw.raw_pattern == ([[2, 1], [3, 0]])).all():
      B = im[0:H:2, 0:W:2, :] # Every alternating value starting from position (0,0) 
      G = im[0:H:2, 1:W:2, :] # Every alternating value starting from position (0,1) 
      R = im[1:H:2, 1:W:2, :] # Every alternating value starting from position (1,1) 
      G_e = im[1:H:2, 0:W:2, :] # Every alternating value starting from position (1,0)
    out = np.concatenate((R, G, B, G_e), axis=2) # Always in R-G-B-G format
    
    return out



In [ ]:
sess = tf.Session()
in_image = tf.placeholder(tf.float32, [None, None, None, 4])
gt_image = tf.placeholder(tf.float32, [None, None, None, 3])
out_image = network(in_image)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
G_loss = tf.reduce_mean(tf.abs(out_image - gt_image))

t_vars = tf.trainable_variables()
lr = tf.placeholder(tf.float32)
# set different optimizer here
#G_opt = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss)
#G_opt = tf.compat.v1.train.AdadeltaOptimizer(learning_rate=1.0).minimize(G_loss)
G_opt = tf.compat.v1.train.RMSPropOptimizer(learning_rate=1e-4).minimize(G_loss)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

# Raw data takes long time to load. Keep them in memory after loaded.
gt_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')
lastepoch = 0
for folder in allfolders:
    lastepoch = np.maximum(lastepoch, int(folder[-4:]))

losses = []
times = []

print('Debug scheme: ', DEBUG, 'Nr. images: ', len(train_ids))

learning_rate = 1e-4
for epoch in range(lastepoch, num_epochs+1):
    if os.path.isdir(result_dir + '%04d' % epoch):
        continue
    cnt = 0
    if epoch > 2000:
        learning_rate = 1e-5

    for ind in np.random.permutation(len(train_ids)):
        # get the path from image id
        train_id = train_ids[ind]
        print(train_id)
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
        in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
        in_fn = os.path.basename(in_path)

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        st = time.time()
        cnt += 1

        if input_images[str(ratio)[0:3]][ind] is None:
            raw = rawpy.imread(in_path)
            input_images[str(ratio)[0:3]][ind] = np.expand_dims(pack_raw(raw), axis=0) * ratio

            gt_raw = rawpy.imread(gt_path)
            im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
            gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)

        # crop
        H = input_images[str(ratio)[0:3]][ind].shape[1]
        W = input_images[str(ratio)[0:3]][ind].shape[2]

        xx = np.random.randint(0, W - ps)
        yy = np.random.randint(0, H - ps)
        input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps, xx:xx + ps, :]
        gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]

        if np.random.randint(2, size=1)[0] == 1:  # random flip
            input_patch = np.flip(input_patch, axis=1)
            gt_patch = np.flip(gt_patch, axis=1)
        if np.random.randint(2, size=1)[0] == 1:
            input_patch = np.flip(input_patch, axis=2)
            gt_patch = np.flip(gt_patch, axis=2)
        if np.random.randint(2, size=1)[0] == 1:  # random transpose
            input_patch = np.transpose(input_patch, (0, 2, 1, 3))
            gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))

        input_patch = np.minimum(input_patch, 1.0)

        _, G_current, output = sess.run([G_opt, G_loss, out_image],
                                        feed_dict={in_image: input_patch, gt_image: gt_patch, lr: learning_rate})
        output = np.minimum(np.maximum(output, 0), 1)
        g_loss[ind] = G_current

        print("%d %d Loss=%.3f Time=%.3f" % (epoch, cnt, np.mean(g_loss[np.where(g_loss)]), time.time() - st))
        losses.append(np.mean(g_loss[np.where(g_loss)]))
        times.append(time.time() - st)
        if epoch % save_freq == 0:
            if not os.path.isdir(result_dir + '%04d' % epoch):
                os.makedirs(result_dir + '%04d' % epoch)

            temp = np.concatenate((gt_patch[0, :, :, :], output[0, :, :, :]), axis=1)
            toimage(temp * 255, high=255, low=0, cmin=0, cmax=255).save(
                result_dir + '%04d/%05d_00_train_%d.jpg' % (epoch, train_id, ratio))

    saver.save(sess, checkpoint_dir + 'model.ckpt')
    np.save(checkpoint_dir + "losses", losses)
    np.save(checkpoint_dir + "times", times)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Debug scheme:  None Nr. images:  161
158


<ipython-input-14-e76536d54557>:48: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]


0 1 Loss=0.207 Time=11.522
33


<ipython-input-6-ef9c8e6e316a>:157: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  strdata = bytedata.tostring()
<ipython-input-14-e76536d54557>:48: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]


0 2 Loss=0.211 Time=1.167
141
0 3 Loss=0.200 Time=1.170
12
0 4 Loss=0.182 Time=1.159
121
0 5 Loss=0.173 Time=1.151
134
0 6 Loss=0.174 Time=1.143
31
0 7 Loss=0.180 Time=1.149
183


<ipython-input-14-e76536d54557>:48: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]


Streaming output truncated to the last 5000 lines.
1703 25 Loss=0.028 Time=0.127
1
1703 26 Loss=0.029 Time=0.131
205
1703 27 Loss=0.029 Time=0.129
72
1703 28 Loss=0.029 Time=0.128
24
1703 29 Loss=0.029 Time=0.128
131
1703 30 Loss=0.029 Time=0.127
102
1703 31 Loss=0.029 Time=0.118
180
1703 32 Loss=0.028 Time=0.127
73
1703 33 Loss=0.029 Time=0.129
204
1703 34 Loss=0.029 Time=0.131
134
1703 35 Loss=0.029 Time=0.130
222
1703 36 Loss=0.029 Time=0.119
85
1703 37 Loss=0.029 Time=0.127
152
1703 38 Loss=0.029 Time=0.119
71
1703 39 Loss=0.029 Time=0.130
75
1703 40 Loss=0.029 Time=0.118
118
1703 41 Loss=0.029 Time=0.114
26
1703 42 Loss=0.029 Time=0.131
33
1703 43 Loss=0.029 Time=0.113
12
1703 44 Loss=0.029 Time=0.130
194
1703 45 Loss=0.029 Time=0.126
148
1703 46 Loss=0.029 Time=0.128
121
1703 47 Loss=0.029 Time=0.132
181
1703 48 Loss=0.029 Time=0.129
161
1703 49 Loss=0.029 Time=0.131
96
1703 50 Loss=0.029 Time=0.115
183
1703 51 Loss=0.029 Time=0.120
66
1703 52 Loss=0.029 Time=0.128
57
1703 53 Los

In [ ]:
# save the model
saver.save(sess, checkpoint_dir + 'model.ckpt')

'./checkpoint/Own_Adadelta_full/model.ckpt'

In [ ]:
# save the losses
np.save(checkpoint_dir + "losses", losses)
np.save(checkpoint_dir + "times", times)